In [1]:
import numpy as np
import csv
import sys
sys.path.append("..")

from Code.VQLSSVM import VQLSSVM
from Code.Utils import prepareDataset

np.set_printoptions(precision=10, suppress=True)

In [2]:
shots: int = 10000
gamma: float = 0.01 # regularization parameter
subsetSize: int = 7
classToFilterOut: int = 2

vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)
# methods = ['COBYLA','SLSQP', 'BFGS', 'L-BFGS-B', 'trust-constr']
methods = ["ADAM", "GD"]
# methods = ["SPSA","ADAM", "GD"]
iterations = 5

In [3]:
def writeToCsv(costs, methods):
    with open('../SimulationResults/costOptimizers.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Iteration'] + methods)
        for i in range(200):
            values = [i]
            for j in range(len(costs)):
                if i >= len(costs[j]):
                    values.append(costs[j][-1])
                else:
                    values.append(costs[j][i])
            writer.writerow(values)

In [8]:
# costsAll = []
# for i in range(iterations):
#     xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset="iris")
#     costs = []
#     print(i,"th iteration")
#     for method in methods:
#         print("Method: " + method)
#         vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=False, verbose=False, iterations = 200, method=method)
#         cost = vqlssvmVectors.getCostHistory()
#         # print("Cost history:")
#         # print(cost)
#         costs.append(cost)
#         print(vqlssvmVectors.accuracy(xTest, yTest))
#     if len(costsAll) == 0:
#         costsAll = np.array(costs)
#     else:
#         if len(costs) < len(costsAll):
#             costs = np.array(costs)
#             costs = np.append(costs, np.full((len(costsAll)-len(costs),), costs[-1]))        
#         if len(costs) > len(costsAll):
#             costs = np.array(costs)
#             costs = costs[:len(costsAll)]
#         costsAll += np.array(costs)

# costsAll *= iterations
# print(costsAll)

#generate list from 0.1 to 0.9 with step 0.1
# vars = np.linspace(0.1, 0.9, num=9)
# print(vars)

costsAll = {}
for method in methods:
    if method == "SPSA":
        for var in vars:
            for var1 in vars:
                costsAll[method+"_"+str(var)+"_"+str(var1)] = []
                for iteration in range(iterations):
                    xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset="iris")
                    print("SPSA with learning_rate", var, "perturbation",var1)
                    vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=False, verbose=False, iterations = 200, method=method,options={"learning_rate":var, "perturbation":var1}, threads=4, jobs=8)
                    cost = vqlssvmVectors.getCostHistory()
                    #take first 200 costs
                    cost = cost[:200]
                    costsAll[method+"_"+str(var)+"_"+str(var1)].append(cost)
    else:
        if method == "GD":
            vars = [0.6, 0.7, 0.8, 0.9]
        else:
            vars = [0.75, 0.9]
        for var in vars:
            costsAll[method+"_"+str(var)] = []
            for iteration in range(iterations):
                xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset="iris")
                print(method, "with learning_rate", var)
                if method == "GD":
                    learning = "learning_rate"
                else:
                    learning = "lr"
                vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=False, verbose=False, iterations = 200, method=method, options={learning:var}, threads=4, jobs=8)
                cost = vqlssvmVectors.getCostHistory()
                cost = cost[:200]
                if costsAll[method+"_"+str(var)] == []:
                    costsAll[method+"_"+str(var)] = cost
                else:
                    costsAll[method+"_"+str(var)] = [a + b for a, b in zip(costsAll[method+"_"+str(var)], cost)]

ADAM with learning_rate 0.1
parametersHadamard prepared
parametersSpecialHadamard prepared
labelVectorCircuit prepared
fixedAnsatzCircuit prepared
controlledFixedAnsatzCircuit prepared
transpiledHadamardCircuits prepared
transpiledSpecialHadamardCircuits prepared
ADAM with learning_rate 0.1569983243899024e-05
parametersHadamard prepared
parametersSpecialHadamard prepared
labelVectorCircuit prepared
fixedAnsatzCircuit prepared
controlledFixedAnsatzCircuit prepared
transpiledHadamardCircuits prepared
transpiledSpecialHadamardCircuits prepared
ADAM with learning_rate 0.127439368469325e-076
parametersHadamard prepared
parametersSpecialHadamard prepared
labelVectorCircuit prepared
fixedAnsatzCircuit prepared
controlledFixedAnsatzCircuit prepared
transpiledHadamardCircuits prepared
transpiledSpecialHadamardCircuits prepared
ADAM with learning_rate 0.1784957356710592e-05
parametersHadamard prepared
parametersSpecialHadamard prepared
labelVectorCircuit prepared
fixedAnsatzCircuit prepared
cont

In [9]:
for key in costsAll:
    costsAll[key] = [x / iterations for x in costsAll[key]]

In [10]:
writeToCsv([costsAll[key] for key in costsAll], [key for key in costsAll]) 